In [1]:
# Basic python packages
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict
import glob
import splitfolders
import cv2

# General machine learning packages
from sklearn.model_selection import train_test_split

# Packages related to images
from PIL import Image
import PIL

# Packages for neural networks
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Embedding
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout
from tensorflow.keras.models import Model

In [2]:
# !pip install --upgrade tensorflow_hub

In [3]:
# Check if GPU works
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
def clean_data(path):
    df = pd.read_csv(path)
    df = df[df['Creator'] != ''] 
    df = df[df['Creator'] != 'anoniem']
    df = df[df['Creator'] != 'onbekend']
    df = df[df['Creator'].notna()]
    df = df.replace('Koning, Cornelis (?-1671)', 'Koning, Cornelis')
    return df

def preprocess_data(df, num_artworks = 10):
    df = df.groupby("Creator").filter(lambda x: len(x) >= num_artworks)
    return df

df = clean_data('../Data/Rijksmuseum/xml_files.csv')
rijksdata = preprocess_data(df, 100)

In [5]:
#Paths to different folders/files
image_dir = "../Data/Rijksmuseum/jpg2/"
split_image_dir = "../Data/Rijksmuseum/jpg2_split_100/"
training_path = "../Data/Rijksmuseum/output_100/train/"
validation_path = "../Data/Rijksmuseum/output_100/val/"
testing_path = "../Data/Rijksmuseum/output_100/test/"

img_size = (200, 200) #Size of the input of the neural networks
IMG_SHAPE = img_size + (3,)
batch_size = 256
n_labels = len(rijksdata.groupby('Creator').count())

In [6]:
labels = rijksdata[['Identifier', 'Creator', 'Date']]
labels
labels[labels['Identifier'] == 'RP-P-1878-A-550']

,Identifier,Creator,Date
29196,RP-P-1878-A-550,"Bleyswyck, François van",1681 - 1746


https://stackoverflow.com/questions/43977463/valueerror-could-not-broadcast-input-array-from-shape-224-224-3-into-shape-2?answertab=scoredesc#tab-top

https://medium.com/analytics-vidhya/multi-task-learning-in-computer-vision-image-classification-ea7b61fb4a0a

https://www.kaggle.com/code/vishnunkumar/mutli-task-learner

In [7]:
list_of_data = []

for fol in os.listdir(split_image_dir):
    for file in os.listdir(os.path.join(split_image_dir, fol)):
        json_dict = {}
        im_path = os.path.join(os.path.join(split_image_dir, fol), file)
        img_arr = np.array(Image.open(im_path).convert('RGB').resize((img_size[0], img_size[1]), Image.ANTIALIAS))
        json_dict['image'] = img_arr
        json_dict['creator'] = labels[labels['Identifier'] == file[:-4]]['Creator'].values[0]
        json_dict['date'] = labels[labels['Identifier'] == file[:-4]]['Date'].values[0]
        list_of_data.append(json_dict)

In [8]:
train = np.array(list_of_data)

x_train = []
y_train = []

for i in range(0, train.shape[0]):
    x_train.append(train[i]['image'])
    y_train.append([train[i]['creator'], train[i]['date']])

In [9]:
# len([item for item in x_train if item.shape == (200, 200)])

In [10]:
x_train_re = np.array(x_train) / 255
x_train_re.reshape(x_train_re.shape[0], img_size[0], img_size[1], 3)

array([[[[0.95686275, 0.94509804, 0.87058824],
         [0.95686275, 0.94509804, 0.87058824],
         [0.95686275, 0.94509804, 0.87058824],
         ...,
         [0.95294118, 0.94901961, 0.87843137],
         [0.95294118, 0.94901961, 0.87843137],
         [0.95294118, 0.94901961, 0.87843137]],

        [[0.95686275, 0.94509804, 0.87058824],
         [0.95686275, 0.94509804, 0.87058824],
         [0.95686275, 0.94509804, 0.87058824],
         ...,
         [0.95294118, 0.94901961, 0.87843137],
         [0.95294118, 0.94901961, 0.87843137],
         [0.95294118, 0.94901961, 0.87843137]],

        [[0.95686275, 0.94509804, 0.87058824],
         [0.95686275, 0.94509804, 0.87058824],
         [0.95686275, 0.94509804, 0.87058824],
         ...,
         [0.95686275, 0.94509804, 0.8745098 ],
         [0.95686275, 0.94509804, 0.8745098 ],
         [0.95686275, 0.94509804, 0.8745098 ]],

        ...,

        [[0.94901961, 0.94509804, 0.86666667],
         [0.94901961, 0.94509804, 0.86666667]

In [11]:
# x_train_re.shape

In [12]:
y_train = np.array(y_train)
class_1 = y_train[:,0]
class_2 = y_train[:,1]
c_1 = len(class_1)
c_2 = len(class_2)
class_2

array(['1530 -  1540', '1528 -  1528', '1554 -  1554', ...,
       '1850 -  1865', '1850 -  1860', '1856 -  1863'], dtype='<U57')

In [13]:
map_1 = {}
for i, j in enumerate(list(set(class_1))):
    map_1[j] = i

map_2 = {}
for i, j in enumerate(list(set(class_2))):
    map_2[j] = i

class_arr1 = []
for x in class_1:
    class_arr1.append(map_1[x])
class_arr1 = np.array(class_arr1)

class_arr2 = []
for x in class_2:
    class_arr2.append(map_2[x])
class_arr2 = np.array(class_arr2)

In [14]:
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#         horizontal_flip = True)  # randomly flip images

# datagen.fit(x_train)

In [15]:
x_train = np.asarray(x_train_re)

In [16]:
def train_model(model, steps_per_epoch=150, epochs=5, validation_steps=70, workers=7, checkpoint_loc=""):
    """
    Trains a given model

    :steps_per_epoch: Amount of batches uploaded per epoch. Cant be higher than +- 200
    :epochs: Amount of times the model trains on the data
    :validation_steps: Amount of batches used for validation. Cant be higher than +- 50
    :workers: Amount of processes used to load the data
    :checkpoint_loc: Place for the model checkpoints to be saved
    :return: The trained model and some training data
    """ 
    # Create a callback that saves the model's weights
#     cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_loc,
#                                                      save_weights_only=True,
#                                                      verbose=1)
#     cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_loc,
#                                                      monitor='acc',
#                                                      save_weights_only=True,
#                                                      save_best_only=True)
#     cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_loc, 
#                                                      monitor='acc', 
#                                                      save_weights_only=True, 
#                                                      save_best_only=True, 
#                                                      mode='max')

#     early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    begin_time = datetime.datetime.now()
    history = model.fit({'main_input': x_train}, {'artist_output': class_arr1, 'year_output': class_arr2}, epochs=epochs, batch_size=128,
              verbose=1,
              validation_split=0.2)
    print(datetime.datetime.now() - begin_time)
    return (model, history)

In [17]:
def plot_history(history):
    #Plots the training data.
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.show() 

# Xception model:

In [18]:
def make_multitask_model(n_labels):
    inputs = tf.keras.layers.Input(shape=[img_size[0], img_size[1], 3], name='main_input')
    main_branch = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/inception_v3/classification/4")(inputs)
    main_branch = tf.keras.layers.Flatten()(main_branch)
    main_branch = tf.keras.layers.Dense(1024, activation='relu')(main_branch)

    class1_pred = Dense(c_1, activation='softmax', name='c1_output')(main_branch)
    class2_pred = Dense(c_2, activation='softmax', name='c2_output')(main_branch)

    model = Model(inputs = inputs, outputs = [class1_pred, class2_pred])
    
    for layer in model.layers[:126]:
        layer.trainable = False
    for layer in model.layers[126:]:
        layer.trainable = True
        
    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                  loss={'c1_output': 'sparse_categorical_crossentropy', 'c2_output': 'sparse_categorical_crossentropy'},
                  loss_weights={'c1_output': .001, 'c2_output': 1.},
                  metrics=['accuracy'])
    
    return model

model = make_multitask_model(n_labels)

In [19]:
model, history = train_model(model, epochs=20, checkpoint_loc="../Model_weights/inception_v3_multitask/")


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [25]:
plot_history(history)

KeyError: 'accuracy'

In [ ]:
# print(STEP_SIZE_TRAIN, STEP_SIZE_VALID)

In [ ]:
# Save model
# model.save("models/_Xception_adam")

# Load model
# model = tf.keras.models.load_model('./models/_ResNet152V2model')
# model.load_weights('../Model_weights/ResNet152V2model/')


# Predict the test set
We also have to predict the real test set

In [ ]:
# def predict_test(model, labels):
#     preds = model.predict(test_generator)
#     preds_cls_idx = preds.argmax(axis=-1)
#     idx_to_cls = {v: k for k, v in train_generator.class_indices.items()}
#     preds_cls = np.vectorize(idx_to_cls.get)(preds_cls_idx)
#     filenames_to_cls = list(zip(test_generator.filenames, preds_cls))
    
#     l = []
#     n = []
#     t = []
#     for p in filenames_to_cls:
#         n.append(p[0].split("\\")[-1][:-4])
#         l.append(p[1])
#         t.append(labels[labels['Identifier'] == p[0].split("\\")[-1][:-4]]['Creator'].values[0])
#     return pd.DataFrame(list(zip(n, l, t)), columns=['img_name', 'predicted label', 'true label'])

# res = predict_test(model, labels)

In [ ]:
# res

In [ ]:
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(res['true label'], res['predicted label'])
# accuracy